In [ ]:
conda install -c anaconda mysql-connector-python

In [2]:
import pandas as pd
import datetime as dt
import hashlib
import mysql.connector

In [ ]:
conn = mysql.connector.connect(host='localhost', 
                                         database='rossmann',
                                         user='root',
                                         password='root')
cursor = conn.cursor(buffered=True)


## Feeding informations in the RECORD_SOURCE_REF table

In [3]:
cursor.execute("""delete from RECORD_SOURCE_REF;""")
cursor.execute("""insert into RECORD_SOURCE_REF  (RECORD_SOURCE, RECORD_PATH) VALUES  (1, ".\\input\\input1.csv"),\
(2, ".\\input\\input2.csv") ,(3, ".\\input\\Geschaefte.csv");""")

### Import and load data  of the stores in the appropriate tables

In [4]:
df = pd.read_csv(f'.\Cleaned Data\stores.csv')

In [ ]:
for line in range(len(df)):
        
    LOAD_DATETIME=dt.datetime.now()
    RECORD_SOURCE = df.loc[line,'Source']
    STORE_ID = df.loc[line,'Store']
    
    #Create  hashing code of hub store  from load_datetime and business key store_id
    string_hash = str(STORE_ID) + str(RECORD_SOURCE)
    STORE_HSH = hashlib.md5(string_hash.encode()).hexdigest()
    
    # insert the data in the table hub_store
    query = "INSERT INTO hub_store (STORE_HSH, LOAD_DATETIME, RECORD_SOURCE, STORE_ID)\
    VALUES ('" + STORE_HSH + "', '" + str(LOAD_DATETIME) + "', '" + str(RECORD_SOURCE) + "', '" + str(STORE_ID) + "');"
    try:
        cursor.execute(query)
    except mysql.connector.IntegrityError as e:
        print("error code:",e)
   
    #  insert data in the satellite sat_store
    
    store_type = df.loc[line, "StoreType"]
    assortment = df.loc[line, "Assortment"]
    LOAD_END_DATETIME=dt.datetime.now()
    string_hash = str(STORE_HSH) + str(RECORD_SOURCE)+ str(LOAD_END_DATETIME)
    SSTORE_HSH=hashlib.md5(string_hash.encode()).hexdigest()
    
    # generating hashdiff code from attributes of sat_store
    string_hash =  str(store_type) + str(assortment)
    diff_hsh = hashlib.md5(string_hash.encode()).hexdigest()
    
      # insert the data in the satellite table sat_store
    query = "INSERT INTO sat_store (SSTORE_HSH, STORE_HSH, DIFF_HSH, LOAD_DATETIME,LOAD_END_DATETIME, RECORD_SOURCE,\
    STORE_TYPE, ASSORTMENT) VALUES ('" + SSTORE_HSH + "','" + STORE_HSH + "','" + diff_hsh + "', '" + str(LOAD_DATETIME) + "',\
    '" + str(LOAD_END_DATETIME)+ "','" + str(RECORD_SOURCE) + "', '" + str(store_type) + "' , '" + str(assortment) + "');"
    try:
        cursor.execute(query)
    except mysql.connector.IntegrityError as e:
        print("error code:",e)
        
        
   #  insert data in the satellite table sat_promo just when the store applies promo2
   
    promo2 = df.loc[line, "Promo2"]
    if (promo2==1):
    
        promo2_since_week = df.loc[line, "Promo2SinceWeek"] 
        promo2_since_year = df.loc[line, "Promo2SinceYear"] 
        promo_interval = df.loc[line, "PromoInterval"] 
        LOAD_DATETIME=dt.datetime.now()
        LOAD_END_DATETIME=dt.datetime.now()
    # generating hashkey of sat_comp
        string_hash=str(STORE_HSH)+ str(RECORD_SOURCE)+ str(LOAD_END_DATETIME)
        promo_hsh=hashlib.md5(string_hash.encode()).hexdigest()
    
     # generating  hashdiff code from attributes of sat_promo
        string_hash =  str(promo2) + str(promo2_since_week)+ str(promo2_since_year)+str(promo_interval)
        diff_hsh = hashlib.md5(string_hash.encode()).hexdigest()
    
        query = "INSERT INTO sat_promo (PROMO_HSH, STORE_HSH, DIFF_HSH, LOAD_DATETIME,LOAD_END_DATETIME, RECORD_SOURCE,\
         PROMO2, PROMO2_SINCE_WEEK, PROMO2_SINCE_YEAR, PROMO_INTERVAL) VALUES ('" + promo_hsh + "','" + STORE_HSH + "',\
         '" + diff_hsh + "', '" + str(LOAD_DATETIME) + "',  '" + str(LOAD_END_DATETIME)+ "','" + str(RECORD_SOURCE) + "',\
          '" + str(promo2) + "' , '" +str(promo2_since_week) + "'  , '" + str(promo2_since_year) +"' , '" + str( promo_interval) + "');"
        try:
            cursor.execute(query)
        except mysql.connector.IntegrityError as e:
            print("error code:",e)
        
    
    
      #  insert data in the satellite table sat_comp
        
    comp_dist = df.loc[line, "CompetitionDistance"]
    month = df.loc[line, "CompetitionOpenSinceMonth"] 
    year = df.loc[line, "CompetitionOpenSinceYear"] 
    LOAD_DATETIME=dt.datetime.now()
    LOAD_END_DATETIME=dt.datetime.now()
    # generating hashkey of sat_comp
    string_hash=str(STORE_HSH)+ str(RECORD_SOURCE)+ str(LOAD_END_DATETIME)
    comp_hsh=hashlib.md5(string_hash.encode()).hexdigest()
    
     # generating  hashdiff code from attributes of sat_comp
    string_hash =  str(comp_dist) + str(month)+ str(year)
    diff_hsh = hashlib.md5(string_hash.encode()).hexdigest()
    
    query = "INSERT INTO sat_comp (COMP_HSH, STORE_HSH, DIFF_HSH, LOAD_DATETIME,LOAD_END_DATETIME, RECORD_SOURCE,\
   COMP_DISTANCE, COMP_OPEN_MONTH, COMP_OPEN_YEAR) VALUES ('" + comp_hsh + "','" + STORE_HSH + "',\
    '" + diff_hsh + "', '" + str(LOAD_DATETIME) + "',  '" + str(LOAD_END_DATETIME)+ "','" + str(RECORD_SOURCE) + "',\
    '" + str(comp_dist) + "' , '" + str( month) + "', '" + str( year) + "');"
    try:
        cursor.execute(query)
    except mysql.connector.IntegrityError as e:
        print("error code:",e)
    
   

In [18]:
print(cursor.execute(" Select * FROM RECORD_SOURCE_REF;"))
pd.read_sql_query(" Select * FROM sat_comp;", con=conn)

None


,COMP_HSH,STORE_HSH,DIFF_HSH,LOAD_DATETIME,LOAD_END_DATETIME,RECORD_SOURCE,COMP_DISTANCE,COMP_OPEN_MONTH,COMP_OPEN_YEAR
0,0033842f1fb2aaf620bded9590681760,14cfdb59b5bda1fc245aadae15b1984a,15ac4e79983b482e2ef9d9d157e5bfc5,2020-09-24 16:38:40,2020-09-24 16:38:40,3,1390,-1,-1
1,004512aa8d860273400c17f4526571cf,24759597b99d21ef84e6c86c29e56aab,b97d49efed73a6aa2e3a3b71f72dc394,2020-09-25 14:28:01,2020-09-25 14:28:01,3,2100,8,2013
2,004d7eafbedeff35f781c5bff5cb2e39,30192e936ba11d0a202097fed8f44b2d,c3c8278f9dc4a54f7dd9d3057e17d75b,2020-09-25 14:28:00,2020-09-25 14:28:00,3,190,9,2011
3,00734bbfeed1aa2d5ec41235c8958866,b151ce4935a3c2807e1dd9963eda16d8,457753f9bd902d3edf26ca4cd65187eb,2020-09-25 14:28:00,2020-09-25 14:28:00,3,10890,4,2005
4,007522cce17a94b9b8667a77c3e08345,414e773d5b7e5c06d564f594bf6384d0,911a13a8cd3728250a9cf2e51492efc3,2020-09-25 14:28:00,2020-09-25 14:28:00,3,2770,3,2008
...,...,...,...,...,...,...,...,...,...
2225,ff4d9a257d9e05e4ee37e49b37eb4451,7b13b2203029ed80337f27127a9f1d28,264e3fcd470dd5fa700aa9182b49247f,2020-09-25 14:28:00,2020-09-25 14:28:00,3,2030,11,2003
2226,ff634864e491b83ca6d110a4bd824ed1,cdd0500dc0ef6682fa6ec6d2e6b577c4,6fabf17581a30928bebb2db0a8c142b3,2020-09-24 16:38:40,2020-09-24 16:38:40,3,4370,-1,-1
2227,ff81bdb83573d25a68d6531a2e45a0dd,1349b36b01e0e804a6c2909a6d0ec72a,7b958058755b826daeb2c2f1ba1700ac,2020-09-24 16:38:40,2020-09-24 16:38:40,3,3200,1,2008
2228,ff828503ac017cc0f629804a1cf25ad7,dfc69ee4c48a622e2bcda2c3d2a219d0,e246a5f7d4d871a97f17649b3f217b1e,2020-09-25 14:28:01,2020-09-25 14:28:01,3,4140,9,2012


## Load the data from the file sales.csv in the appropriate tables in the datavault model

In [6]:
df = pd.read_csv(f'.\Cleaned Data\sales.csv')

C:\Users\makou\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
df.head(10)

,Unnamed: 0,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Source,Customers,Sales
0,1,1,4,2015-09-17,1,1,0,0,1,-1,-1
1,2,3,4,2015-09-17,1,1,0,0,1,-1,-1
2,3,7,4,2015-09-17,1,1,0,0,1,-1,-1
3,4,8,4,2015-09-17,1,1,0,0,1,-1,-1
4,5,9,4,2015-09-17,1,1,0,0,1,-1,-1
5,6,10,4,2015-09-17,1,1,0,0,1,-1,-1
6,7,11,4,2015-09-17,1,1,0,0,1,-1,-1
7,8,12,4,2015-09-17,1,1,0,0,1,-1,-1
8,9,13,4,2015-09-17,1,1,0,0,1,-1,-1
9,10,14,4,2015-09-17,1,1,0,0,1,-1,-1


In [7]:

for line in range(len(df)):
    
    # loading data for hub_day
    date=df.loc[line,'Date']
    record_source=df.loc[line,'Source']
  
    # generating hashkey of hub_day always when it is a new entry
    
    # check if the date already exists to avoid duplicated primary keys in hub_day and sat_day before inserting new entries
    query=cursor.execute("SELECT DAY_HSH FROM hub_day WHERE DATE = '" + str(date) + "';")
    result=cursor.fetchall()
    if not result:
        string_hash=str(date)+ str(record_source)
        day_hsh= hashlib.md5(string_hash.encode()).hexdigest()
        LOAD_DATETIME=dt.datetime.now()
        query = "INSERT INTO hub_day (DAY_HSH, LOAD_DATETIME, RECORD_SOURCE, DATE)\
        VALUES ('" + day_hsh + "', '" + str(LOAD_DATETIME) + "', '" + str(record_source) + "', '" + str(date) + "');"
        try:
            cursor.execute(query)
        except mysql.connector.IntegrityError as e:
            print("error code:",e)
        
         # loading data in the satellite sat_day
            
        day_week= df.loc[line,"DayOfWeek"]
        LOAD_DATETIME=dt.datetime.now()
        LOAD_END_DATETIME=dt.datetime.now()
     # generating hashkey of sat_day
        string_hash=str(day_hsh)+str(record_source)+str(LOAD_END_DATETIME)
        sat_day_hsh= hashlib.md5(string_hash.encode()).hexdigest()
    
      # generating  hashdiff code from attributes of sat_day
        string_hash =  str(day_week) + str(record_source) 
        diff_hsh = hashlib.md5(string_hash.encode()).hexdigest()
    
        query = "INSERT INTO sat_day (SAT_DAY_HSH,DAY_HSH,DIFF_HSH, LOAD_DATETIME,LOAD_END_DATETIME, RECORD_SOURCE, DAY_OF_WEEK)\
             VALUES ('" +sat_day_hsh + "','" + day_hsh + "','" + diff_hsh + "', '" + str(LOAD_DATETIME) + "',\
            '" + str(LOAD_END_DATETIME) + "','" + str(record_source) + "', '" + str(day_week) + "');"
        try:
            cursor.execute(query)
        except mysql.connector.IntegrityError as e:
            print("error code:",e)
    #else: 
    #    print(' This date entry already exists in the hub_day')
        
        
        
    
         
    #loading data in the link link_day
    
    store_id= df.loc[line,'Store']
    store_hsh=pd.read_sql_query("select STORE_HSH from hub_store where STORE_ID= '" + str(store_id)+"';", con=conn).iloc[0,0]
    day_hsh= pd.read_sql_query("select DAY_HSH from hub_day where DATE= '" + str(date)+"';" , con=conn).iloc[0,0]
    
    LOAD_DATETIME=dt.datetime.now()
    # generating hashkey of link_day
    string_hash=str(store_hsh)+ str(day_hsh)+str(record_source)
    link_day_hsh= hashlib.md5(string_hash.encode()).hexdigest()
    
    query = "INSERT INTO link_day (LINK_DAY_HSH,STORE_HSH,DAY_HSH, LOAD_DATETIME, RECORD_SOURCE)\
    VALUES ('" + link_day_hsh + "', '" + store_hsh + "', '" + str(day_hsh) + "', '" + str(LOAD_DATETIME) + "', \
    '" + str(record_source) + "');"
    try:
        cursor.execute(query)
    except mysql.connector.IntegrityError as e:
        print("error code:",e)
 

    #loading data in the satellite  sat_day_facts
    opens=df.loc[line,'Open']
    promo=df.loc[line,'Promo']
    state_hol=df.loc[line,'StateHoliday']
    school_hol=df.loc[line,'SchoolHoliday']
    sales=df.loc[line,'Sales']
    customers=df.loc[line,'Customers']
    LOAD_DATETIME=dt.datetime.now()
    LOAD_END_DATETIME=dt.datetime.now()
    
     # generating hashkey of sat_day_facts
    string_hash=str(link_day_hsh)+str(record_source)+ str(LOAD_END_DATETIME)
    sat_dayf_hsh= hashlib.md5(string_hash.encode()).hexdigest()
    
     # generating  hashdiff code from attributes of sat_day_facts
    string_hash =  str(opens) + str(state_hol) + str(school_hol) + str(promo) + str(customers) + str(sales)
    diff_hsh = hashlib.md5(string_hash.encode()).hexdigest()
    
    query = "INSERT INTO sat_day_facts (SAT_DAYF_HSH, LINK_DAY_HSH, DIFF_HSH, LOAD_DATETIME,LOAD_END_DATETIME, RECORD_SOURCE,\
   OPEN, STATE_HOLIDAY, SCHOOL_HOLIDAY, PROMO, CUSTOMERS, SALES) VALUES ('" + sat_dayf_hsh + "','" + link_day_hsh + "',\
    '" + diff_hsh + "', '" + str(LOAD_DATETIME) + "',  '" + str(LOAD_END_DATETIME)+ "','" + str(RECORD_SOURCE) + "',\
    '" + str(opens) + "' , '" + str( state_hol) + "', '" + str( school_hol) + "' , '" + str( promo) + "',\
      '" + str( customers) + "', '" + str( sales) + "');"
    try:
        cursor.execute(query)
    except mysql.connector.IntegrityError as e:
        print("error code:",e)
    
  

   

In [ ]:
pd.read_sql_query("select * from hub_day;" , con=conn)

In [20]:
pd.read_sql_query("select * from sat_day_facts LIMIT 20;" , con=conn)

,SAT_DAYF_HSH,LINK_DAY_HSH,DIFF_HSH,LOAD_DATETIME,LOAD_END_DATETIME,RECORD_SOURCE,OPEN,STATE_HOLIDAY,SCHOOL_HOLIDAY,PROMO,CUSTOMERS,SALES
0,00000776a4d6e2beb3c817ef37a1ce14,dadbb64600594712efdffb71f8b1d70c,9b5ea601efad6c7bf5e7b4c4f3499bf2,2020-09-24 17:49:39,2020-09-24 17:49:39,3,1,0,0,1,641,7846
1,0000265c25568c9627f09fc46a083863,abc028351f0f2edbf7bc116fca014589,fd1fc1ad353455c0841df80423ccd461,2020-09-24 17:04:26,2020-09-24 17:04:26,3,1,0,0,1,493,3356
2,00002cd0958f54ac24af7334908f672e,6311280aa9c8f0030d1d2b29a9077285,6e42c1c679ad8bcb06411e136716e101,2020-09-24 17:53:02,2020-09-24 17:53:02,3,1,0,0,1,984,10334
3,0000443c0f4fe0abe44fbae8c153aaf2,3c43b2d20101b64d0b29e1f5e16fc864,014344289788aeb42e257c806cab6bd8,2020-09-24 17:46:46,2020-09-24 17:46:46,3,1,0,0,0,608,4344
4,00004ba0774427922fdcf3d76d505cba,3d92c0d717e1b88ab881cbf697753a4e,983225edfffd12c6204dcf00ddee3891,2020-09-24 16:46:19,2020-09-24 16:46:19,3,1,0,0,0,1708,14155
5,00005fda5bcf61aff2cbe28b58c339ad,b1f065655db62ada087342b6a7fe5e50,d66583c10e4db56a600895f38d07e6fe,2020-09-24 17:34:02,2020-09-24 17:34:02,3,1,0,0,1,847,8931
6,0000601c48109deac83e331283720ded,8fe63a112094e99cd4ae4beec9a7e278,6a0fb0f562481e16926f76a19fb6c396,2020-09-24 16:57:11,2020-09-24 16:57:11,3,1,0,0,1,1023,10646
7,00006dc92de68965e9196d85ace8c800,738882b9829cd74074fbcde977dd5874,06a0b8742d48bf1054fc782b00bf97d2,2020-09-24 16:58:43,2020-09-24 16:58:43,3,1,0,1,1,848,9878
8,00006f92eb6042b4369a3965fe446323,7a52df489e1bcb5c5c4364ea4897c1a7,62c9aa42c69ea18cf6959d391d91dfa7,2020-09-24 18:10:12,2020-09-24 18:10:12,3,1,0,0,1,980,10389
9,00008793dfefb295f6c2a2b501d1206e,a712f6967215dc6cec6f18853f52bb37,4718e7a51122643cecb76422d4c071c0,2020-09-24 17:53:08,2020-09-24 17:53:08,3,1,0,0,0,597,5885


In [6]:
cursor.close()
conn.commit()
conn.close()